# PETsARD Adapter 測試

這個 notebook 用來測試各個 Adapter 的功能，按照 test.ipynb 的設定進行調整。

## LoaderAdapter 測試

In [1]:
from petsard.adapter import LoaderAdapter

# 測試 benchmark 資料載入
loader_adpt = LoaderAdapter(config={"filepath": "benchmark://adult-income"})

# 執行載入
loader_adpt.run(input={})

# 取得結果
data = loader_adpt.get_result()
meta = loader_adpt.get_metadata()

print(f"data: {type(data)}, meta: {type(meta)}")
print(f"data shape: {data.shape}")
print(f"Schema id: {meta.id}")
print(f"Schema name: {meta.name}")

data: <class 'pandas.core.frame.DataFrame'>, meta: <class 'petsard.metadater.metadata.Schema'>
data shape: (48842, 15)
Schema id: adult-income
Schema name: adult-income.csv


In [2]:
# 測試本地檔案載入
loader_adpt_local = LoaderAdapter(config={"filepath": "benchmark/adult-income.csv"})

loader_adpt_local.run(input={})

data_local = loader_adpt_local.get_result()
meta_local = loader_adpt_local.get_metadata()

print(f"data: {type(data_local)}, meta: {type(meta_local)}")
print(f"data shape: {data_local.shape}")

data: <class 'pandas.core.frame.DataFrame'>, meta: <class 'petsard.metadater.metadata.Schema'>
data shape: (48842, 15)


# Just simplify

In [3]:
data = data.loc[0:99, :]
data.shape

(100, 15)

## SplitterAdapter 測試

In [4]:
from petsard.adapter import SplitterAdapter

# 測試資料分割 - 使用與 test.ipynb 相同的配置
splitter_adpt = SplitterAdapter(config={"num_samples": 5, "train_split_ratio": 0.8})

# 準備輸入資料
splitter_input = {"data": data, "metadata": meta, "exclude_index": []}

splitter_adpt.run(input=splitter_input)

# 取得分割結果 - 使用第一次分割
split_results = splitter_adpt.get_result()
split_meta = splitter_adpt.get_metadata()

# 取得第一次分割的訓練集和測試集 - 限制只能取第一個
train_data = split_results["train"]
test_data = split_results["validation"]

print(f"train_data: {type(train_data)}, test_data: {type(test_data)}")
print(f"train_data shape: {train_data.shape}, test_data shape: {test_data.shape}")

train_data: <class 'pandas.core.frame.DataFrame'>, test_data: <class 'pandas.core.frame.DataFrame'>
train_data shape: (80, 15), test_data shape: (20, 15)


In [5]:
# 測試自定義資料分割
splitter_adpt_custom = SplitterAdapter(
    config={
        "method": "custom_data",
        "filepath": {
            "ori": "benchmark://adult-income_ori",
            "control": "benchmark://adult-income_control",
        },
    }
)

# 自定義資料不需要輸入資料
splitter_adpt_custom.run(input={"exclude_index": []})

custom_split_result = splitter_adpt_custom.get_result()
print(f"Custom split result keys: {custom_split_result.keys()}")

Custom split result keys: dict_keys(['train', 'validation'])


## PreprocessorAdapter 測試

In [6]:
from petsard.adapter import PreprocessorAdapter

# 測試預設預處理
preproc_adpt = PreprocessorAdapter(config={"method": "default"})

preproc_input = {"data": train_data, "metadata": meta}

preproc_adpt.run(input=preproc_input)

default_preproc_data = preproc_adpt.get_result()
preproc_meta = preproc_adpt.get_metadata()

print(f"default_preproc_data shape: {default_preproc_data.shape}")

default_preproc_data shape: (54, 15)


In [7]:
# 測試僅處理缺失值
preproc_adpt_missing = PreprocessorAdapter(
    config={
        "method": "custom",
        "config": {
            "missing": {
                "age": "missing_mean",
            },
        },
        "sequence": ["missing"],
    }
)

preproc_adpt_missing.run(input=preproc_input)

missing_preproc_data = preproc_adpt_missing.get_result()

print(f"missing_preproc_data shape: {missing_preproc_data.shape}")
print("Age column after missing value processing:")
print(missing_preproc_data["age"].head(10))

missing_preproc_data shape: (80, 15)
Age column after missing value processing:
0    38
1    28
2    44
3    18
4    34
5    29
6    65
7    58
8    48
9    43
Name: age, dtype: int64


In [8]:
# 測試僅處理離群值
preproc_adpt_outlier = PreprocessorAdapter(
    config={
        "method": "custom",
        "config": {
            "outlier": {
                "age": "outlier_zscore",
            },
        },
        "sequence": ["outlier"],
    }
)

preproc_adpt_outlier.run(input=preproc_input)

outlier_preproc_data = preproc_adpt_outlier.get_result()

print(f"outlier_preproc_data shape: {outlier_preproc_data.shape}")
print("Age column after outlier processing:")
print(outlier_preproc_data["age"].head(10))

outlier_preproc_data shape: (54, 15)
Age column after outlier processing:
0    38
1    28
2    18
3    34
4    29
5    58
6    43
7    43
8    34
9    34
Name: age, dtype: int64


In [9]:
# 測試僅編碼處理
preproc_adpt_encoder = PreprocessorAdapter(
    config={
        "method": "custom",
        "config": {
            "encoder": {
                "workclass": "encoder_onehot",
            },
        },
        "sequence": ["encoder"],
    }
)

preproc_adpt_encoder.run(input=preproc_input)

encoder_preproc_data = preproc_adpt_encoder.get_result()

print(f"encoder_preproc_data shape: {encoder_preproc_data.shape}")
print("Workclass columns after encoding:")
workclass_cols = [
    col for col in encoder_preproc_data.columns if col.startswith("workclass_")
]
print(encoder_preproc_data[workclass_cols].head(10))

encoder_preproc_data shape: (80, 19)
Workclass columns after encoding:
   workclass_Local-gov  workclass_Private  workclass_Self-emp-inc  \
0                  0.0                1.0                     0.0   
1                  1.0                0.0                     0.0   
2                  0.0                1.0                     0.0   
3                  0.0                0.0                     0.0   
4                  0.0                1.0                     0.0   
5                  0.0                0.0                     0.0   
6                  0.0                1.0                     0.0   
7                  0.0                0.0                     0.0   
8                  0.0                1.0                     0.0   
9                  0.0                1.0                     0.0   

   workclass_Self-emp-not-inc  workclass_State-gov  
0                         0.0                  0.0  
1                         0.0                  0.0  
2         

In [10]:
# 測試僅縮放處理
preproc_adpt_scaler = PreprocessorAdapter(
    config={
        "method": "custom",
        "config": {
            "scaler": {
                "age": "scaler_minmax",
            },
        },
        "sequence": ["scaler"],
    }
)

preproc_adpt_scaler.run(input=preproc_input)

scaler_preproc_data = preproc_adpt_scaler.get_result()

print(f"scaler_preproc_data shape: {scaler_preproc_data.shape}")
print("Age column after scaling:")
print(scaler_preproc_data["age"].head(10))

scaler_preproc_data shape: (80, 15)
Age column after scaling:
0    0.381818
1    0.200000
2    0.490909
3    0.018182
4    0.309091
5    0.218182
6    0.872727
7    0.745455
8    0.563636
9    0.472727
Name: age, dtype: float64


## SynthesizerAdapter 測試

In [11]:
from petsard.adapter import SynthesizerAdapter

# 測試合成器 - 使用預設方法
synth_adpt = SynthesizerAdapter(config={"method": "default"})

synth_input = {"data": default_preproc_data, "metadata": preproc_meta}

synth_adpt.run(input=synth_input)

syn_data = synth_adpt.get_result()

print(f"syn_data shape: {syn_data.shape}")
print(f"syn_data type: {type(syn_data)}")

syn_data shape: (54, 15)
syn_data type: <class 'pandas.core.frame.DataFrame'>


## PostprocessorAdapter 測試

In [12]:
from petsard.adapter import PostprocessorAdapter


# 測試後處理
postproc_adpt = PostprocessorAdapter(config={"method": "default"})

postproc_input = {
    "data": syn_data,
    "preprocessor": preproc_adpt.processor,
}

postproc_adpt.run(input=postproc_input)

postproc_data = postproc_adpt.get_result()

print(f"postproc_data shape: {postproc_data.shape}")
print(f"postproc_data type: {type(postproc_data)}")

postproc_data shape: (54, 15)
postproc_data type: <class 'pandas.core.frame.DataFrame'>


In [13]:
syn_data.head(6)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,1.565035,0.842944,-0.017571,0.471425,0.329513,0.405965,0.608966,0.620807,0.474309,0.253546,-0.354941,-0.159976,0.292883,0.214628,0.512787
1,0.110907,0.977954,0.105864,0.583357,-1.012026,0.214973,0.817060,0.129007,0.885031,0.575418,-0.354941,-0.159976,-0.248389,0.856323,0.088782
2,1.664051,0.662265,-0.048130,0.557299,-0.637325,0.277570,0.955109,0.670856,0.386097,0.811941,-0.354941,-0.159976,-0.313965,0.414228,0.323169
3,-0.904587,0.605662,0.345490,0.406794,-0.957813,0.402990,0.251461,0.655380,0.135938,0.276309,-0.354941,-0.159976,-0.027601,0.887132,0.556160
4,1.174652,0.919621,-1.384106,0.755349,1.685963,0.425807,0.415748,0.054421,0.610286,0.243771,-0.354941,-0.159976,0.035073,0.439762,0.907129
5,-1.263698,0.391152,0.275300,0.072462,-1.172983,0.193654,0.758696,0.567789,0.159096,0.075747,-0.354941,-0.159976,-0.692459,0.544846,0.225947


In [14]:
postproc_data.head(6)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,59,Self-emp-inc,190045,Some-college,10,Married-civ-spouse,Adm-clerical,Not-in-family,White,Male,0,0,43,United-States,<=50K
1,39,State-gov,202868,Some-college,7,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,37,United-States,<=50K
2,61,Private,186870,Some-college,8,Married-civ-spouse,Tech-support,Not-in-family,White,Female,0,0,36,United-States,<=50K
3,25,Private,227762,Some-college,7,Married-civ-spouse,Exec-managerial,Not-in-family,White,Male,0,0,39,United-States,<=50K
4,54,Local-gov,48078,Masters,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
5,20,Private,220470,HS-grad,6,Married-civ-spouse,Sales,Not-in-family,White,Male,0,0,31,United-States,<=50K


## ConstrainerAdapter 測試

In [15]:
from petsard.adapter import ConstrainerAdapter


# 測試約束器 - 空配置
constr_adpt = ConstrainerAdapter(config={})

constr_input = {"data": postproc_data}

constr_adpt.run(input=constr_input)

cnst_data = constr_adpt.get_result()

print(f"cnst_data shape: {cnst_data.shape}")
print(f"cnst_data type: {type(cnst_data)}")

cnst_data shape: (54, 15)
cnst_data type: <class 'pandas.core.frame.DataFrame'>


In [16]:
# 測試約束器 - 使用 resample_until_satisfy
satisfy_data = constr_adpt.constrainer.resample_until_satisfy(
    data=postproc_data,
    target_rows=postproc_data.shape[0],
    synthesizer=synth_adpt.synthesizer,
    max_trials=300,
    sampling_ratio=10.0,
    verbose_step=10,
)

print(f"satisfy_data shape: {satisfy_data.shape}")

satisfy_data shape: (54, 15)


## EvaluatorOperator 測試

In [ ]:
from petsard.adapter import EvaluatorAdapter

# 測試評估器 - 預設方法
eval_adpt = EvaluatorAdapter(config={"method": "default"})

eval_input = {
    "data": {
        "ori": train_data,
        "control": test_data,
        "syn": satisfy_data,
    },
    "schema": meta,
}

eval_adpt.run(input=eval_input)

eval_result = eval_adpt.get_result()

print(f"Evaluation result keys: {eval_result.keys()}")
print("\nGlobal evaluation results:")
print(eval_result["global"].head(1))
print("\nColumnwise evaluation results:")
print(eval_result["columnwise"].head(1))
print("\nPairwise evaluation results:")
print(eval_result["pairwise"].head(1))

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 2350.36it/s]|
Column Shapes Score: 88.16%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:00<00:00, 1038.40it/s]|
Column Pair Trends Score: 59.28%

Overall Score (Average): 73.72%

Evaluation result keys: dict_keys(['global', 'columnwise', 'pairwise'])

Global evaluation results:
        Score  Column Shapes  Column Pair Trends
result   0.74           0.88                0.59

Columnwise evaluation results:
          Property        Metric     Score
age  Column Shapes  KSComplement  0.866667

Pairwise evaluation results:
                         Property                 Metric     Score  \
age workclass  Column Pair Trends  ContingencySimilarity  0.602315   

               Real Correlation  Synthetic Correlation Error  
age workclass               NaN                    NaN  None  


In [ ]:
# 測試評估器 - MLUtility Classification
eval_adpt_mlutility = EvaluatorAdapter(
    config={
        "method": "mlutility-classification",
        "target": "income",
    }
)

eval_input_anon = {
    "data": {
        "ori": train_data,
        "control": test_data,
        "syn": satisfy_data,
    },
    "schema": meta,
}

eval_adpt_mlutility.run(input=eval_input_anon)

eval_result_mlutility = eval_adpt_mlutility.get_result()

print(eval_result_mlutility["global"].head(1))

The cardinality of the column workclass is too high. Ori: Over row numbers 80, column cardinality 6. Syn: Over row numbers 54, column cardinality 6. The column workclass is removed.
The cardinality of the column education is too high. Ori: Over row numbers 80, column cardinality 12. Syn: Over row numbers 54, column cardinality 7. The column education is removed.
The cardinality of the column marital-status is too high. Ori: Over row numbers 80, column cardinality 6. Syn: Over row numbers 54, column cardinality 5. The column marital-status is removed.
The cardinality of the column occupation is too high. Ori: Over row numbers 80, column cardinality 14. Syn: Over row numbers 54, column cardinality 11. The column occupation is removed.
The cardinality of the column relationship is too high. Ori: Over row numbers 80, column cardinality 5. Syn: Over row numbers 54, column cardinality 5. The column relationship is removed.


   ori_mean  ori_std  syn_mean  syn_std  diff
0      0.94     0.04      0.76     0.04 -0.18


## DescriberOperator 測試

In [19]:
from petsard.adapter import DescriberAdapter

# 測試描述器
desc_op = DescriberAdapter(config={"method": "default"})

desc_input = {"data": {"data": satisfy_data}}

desc_op.run(input=desc_input)

desc_result = desc_op.get_result()

print(f"Description result type: {type(desc_result)}")
if isinstance(desc_result, dict):
    print(f"Description result keys: {desc_result.keys()}")

    print(desc_result["global"].head(1))
    print(desc_result["columnwise"].head(1))
    print(desc_result["pairwise"].head(1))

Description result type: <class 'dict'>
Description result keys: dict_keys(['global', 'columnwise', 'pairwise'])
   row_count  col_count  na_count
0         54         15         0
     mean  median    std   min   max  kurtosis  skew     q1    q3  na_count  \
age  39.3    37.0  15.89  17.0  69.0     -1.27  0.29  25.25  53.5         0   

    nunique  
age    <NA>  
  column1 column2  corr
0     age     age   1.0


## ReporterOperator 測試

In [20]:
from petsard.adapter import ReporterAdapter

# 測試報告器 - Save Data
reporter_op_data = ReporterAdapter(
    config={
        "method": "save_data",
        "source": "Postprocessor",
    }
)

report_data_input = {"data": {("Postprocessor", "exp1"): satisfy_data}}

reporter_op_data.run(input=report_data_input)

report_result_data = reporter_op_data.get_result()

print(f"Save data report result: {report_result_data}")

Save data report result: {'Postprocessor[exp1]':     age         workclass  fnlwgt     education  educational-num  \
0    59           Private  196151       HS-grad               13   
1    44           Private   80450     Bachelors               12   
2    32           Private  162545  Some-college               13   
3    44           Private  246062       HS-grad               12   
4    32           Private  143308          10th                7   
5    20           Private  220470       HS-grad                6   
6    30           Private  123694       HS-grad               13   
7    68         Local-gov  114937       HS-grad               13   
8    25           Private  227762  Some-college                7   
9    21           Private   98587  Some-college                6   
10   47           Private  363872     Assoc-voc               10   
11   17                 ?  221962       HS-grad                9   
12   51           Private  104484  Some-college               11   

In [21]:
# 測試報告器 - Save Report
reporter_op_report = ReporterAdapter(
    config={
        "method": "save_report",
        "granularity": "global",
    }
)

report_report_input = {
    "data": {
        ("Evaluator", "eval1_[global]"): desc_result["global"],
    }
}

reporter_op_report.run(input=report_report_input)

report_result_report = reporter_op_report.get_result()

print(f"Save report result: {report_result_report}")

Save report result: {'[global]':       full_expt_name Evaluator  eval1_row_count  eval1_col_count  \
0  Evaluator[global]  [global]               54               15   

   eval1_na_count  
0               0  }


## 總結

這個 notebook 展示了如何使用各個 Operator 類別來執行 PETsARD 的功能，按照 test.ipynb 的設定進行調整：

1. **LoaderOperator**: 載入資料和元資料
2. **SplitterOperator**: 分割資料為訓練集和測試集
3. **PreprocessorOperator**: 資料預處理（缺失值、離群值、編碼、縮放）
4. **SynthesizerOperator**: 合成資料
5. **PostprocessorOperator**: 後處理合成資料
6. **ConstrainerOperator**: 應用資料約束
7. **EvaluatorOperator**: 評估合成資料品質
8. **DescriberOperator**: 描述資料特性
9. **ReporterOperator**: 生成報告

每個 Operator 都遵循相同的模式：
- 使用配置初始化
- 準備輸入資料
- 執行 `run()` 方法
- 使用 `get_result()` 取得結果
- 使用 `get_metadata()` 取得元資料（如果適用）

這種設計讓每個模組都可以獨立測試，不需要依賴 Status 物件。